In [190]:
!pip install pandas
!pip install json
!pip install numpy

ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


In [191]:
import pandas as pd
import json
import numpy as np
from collections import Counter
import re

In [192]:
df = pd.read_csv("data_travaux/travaux-engages-metropole-lyon.csv", sep=";")
print(df.describe())
display(df)

             numero    code_insee            gid
count  1.319000e+03   1319.000000    1319.000000
mean   2.024778e+08  69258.603487  195480.064443
std    9.287741e+04    121.778735   10836.950822
min    2.018121e+08  69003.000000   74592.000000
25%    2.025078e+08  69157.500000  196242.500000
50%    2.025110e+08  69271.000000  198649.000000
75%    2.025124e+08  69383.000000  199952.500000
max    2.025134e+08  69389.000000  200753.000000


,numero,intervenant,nature_chantier,nature_travaux,etat,date_debut,date_fin,mesures_police,last_update,adresse,commune,code_insee,contact_tel,contact_mail,contact_url,gid
0,202206858,LYON - Direction De La Construction,Travaux sur une parcelle en limite d'espace pu...,Renfort chaussée en vue de la pose d'une grue,Ouvert,2024-06-03,2028-12-31,NaN,2025-02-07 09:02:15.051362+01:00,"10 rue Neyret, 69001 Rue des Tables Claudienn...",Lyon 1er,69381,NaN,NaN,NaN,183025
1,202512731,ORANGE SA,"Branchements réseau < 25 ml TELECOM, RMT ET VIDEO","Intervention sur des chambres, boites, armoire...",Validé,2025-12-24,2026-01-07,"Circulation alternée, Restriction de jour",2025-12-16 15:02:03.127680+01:00,Face au 10 Route du Bruissin (Francheville),Francheville,69089,NaN,NaN,NaN,199942
2,202512882,DCE / RT / Rive Gauche,Émergences et autres équipements d'assainissement,Remise à niveau / entretien des tampons et éme...,Validé,2025-12-15,2026-01-16,NaN,2025-12-04 12:02:37.539147+01:00,"Avenue d'Oschatz, de Rue Gaston Monmousseau a...",Venissieux,69259,04 28 67 55 00,NaN,NaN,199946
3,202512887,ORANGE SA,"Branchements réseau < 25 ml TELECOM, RMT ET VIDEO",Réparation de branchement < 25 ml (télécom ou ...,Validé,2025-12-12,2026-01-19,NaN,2025-12-04 12:02:38.122193+01:00,à côté 3 Chemin de Chantegrillet (Francheville),Francheville,69089,NaN,NaN,NaN,199947
4,202513013,DCE / RT / Rive Droite Sud,"Construction, renouvellement et réparation rés...",Réparation de canalisation d'assainissement,Validé,2025-12-08,2025-12-19,NaN,2025-12-04 12:02:38.969904+01:00,35 Rue Pierre Dupont (Lyon 1er Arrondissement),Lyon 1er,69381,04 28 67 55 20,NaN,NaN,199954
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1314,202512873,RCU Centre Métropole - ELM (Dalkia),"Construction, renouvellement et réparation rés...",Construction de réseau sans branchement (Chauf...,Validé,2025-12-02,2025-12-24,NaN,2025-12-16 15:02:03.235650+01:00,"Du 16 Rue du 8 Mai 1945, au 104 Place du 8 Mai...",Saint Priest,69290,NaN,NaN,NaN,199822
1315,202301053,AOMTL (SYTRAL Mobilités),Aménagement pour les transports en commun,Création / aménagement de voie Bus / Tram,Ouvert,2023-02-07,2026-08-31,NaN,2023-02-07 18:07:40.672980+01:00,- Rue Rue Florian (Villeurbanne) : De Avenue A...,Villeurbanne,69266,NaN,NaN,NaN,135907
1316,202301060,AOMTL (SYTRAL Mobilités),Aménagement pour les transports en commun,Création / aménagement de voie Bus / Tram,Ouvert,2023-01-23,2026-08-31,NaN,2023-02-07 18:07:10.246549+01:00,Tracé T6N du carrefour Zola/Verlaine Bourgey a...,Villeurbanne,69266,NaN,NaN,NaN,135939
1317,202301064,AOMTL (SYTRAL Mobilités),Aménagement pour les transports en commun,Création / aménagement de voie Bus / Tram,Ouvert,2023-01-23,2026-08-31,NaN,2023-02-07 18:07:31.539607+01:00,Traversée futur Parc - Usine ACI Villeurbanne ...,Villeurbanne,69266,NaN,NaN,NaN,135921


In [193]:
# --- 1. GÉNÉRATION DU FICHIER POINTS ROUGES (Coordonnées + Description) ---

with open('data_coord/points-rouges-200046977.geojson', 'r') as f:
    geojson_data = json.load(f)

# Extraction des coordonnées (longitude, latitude) et des propriétés
points_rouges_data = []
for feature in geojson_data['features']:
    props = feature['properties']
    coords = feature['geometry']['coordinates']
    
    points_rouges_data.append({
        'longitude': coords[0],
        'latitude': coords[1],
        'commune_insee': props.get('commune'), 
        'description': props.get('description') 
    })

df_points_rouges_final = pd.DataFrame(points_rouges_data)

# Sauvegarde du premier fichier
df_points_rouges_final.to_csv('points_rouges_lyon_complet.csv', index=False)


# --- 2. GÉNÉRATION DU FICHIER RÉPONSES (Questionnaire + Lieu) ---

# Chargement du CSV original
df_reponses = pd.read_csv('data_coord/reponses-epci-200046977.csv', low_memory=False, quotechar='"')

# Sélection des colonnes stratégiques selon la notice [cite: 86, 88, 90]
cols_reponses = [
    'insee',        # Lieu (Code INSEE) [cite: 21, 29]
    'q6',           # Fréquence de pratique [cite: 86]
    'q14',          # Sentiment de sécurité [cite: 86]
    'q18',          # Dangerosité carrefours [cite: 88]
    'q21',          # État de l'entretien [cite: 88]
    'q28',          # Conflit stationnement motorisé [cite: 88]
    'q34_texte',    # Les 3 priorités d'amélioration [cite: 88]
    'q35',          # Commentaire libre (NLP) [cite: 88]
    'q38_texte',    # Situations de violence vécues [cite: 90]
    'score'         # Note globale de la commune [cite: 30]
]

# Filtrage et nettoyage
df_reponses_final = df_reponses[cols_reponses].copy()
df_reponses_final['q35'] = df_reponses_final['q35'].str.replace(r'\r+|\n+', ' ', regex=True).fillna('')

# Sauvegarde du second fichier
df_reponses_final.to_csv('reponses_questionnaire_lyon.csv', index=False)

print("Les deux fichiers ont été générés avec succès :")
print("- points_rouges_lyon_complet.csv (Données cartographiques)")
print("- reponses_questionnaire_lyon.csv (Données thématiques et profils)")

Les deux fichiers ont été générés avec succès :
- points_rouges_lyon_complet.csv (Données cartographiques)
- reponses_questionnaire_lyon.csv (Données thématiques et profils)


In [194]:
# --- 1. CONFIGURATION RÉFÉRENTIELLE ---
# On centralise ici la connaissance métier : Urgence et Facilité par type de problème
CONFIG_PROBLEMES = {
    'Infrastructure manquante': {'mots': r'piste|manque|absence|discontinuité|coupure', 'urgence': 3, 'facilite': 3},
    'Carrefours dangereux': {'mots': r'carrefour|intersection|rond-point|traversée', 'urgence': 3, 'facilite': 3},
    'Vitesse excessive': {'mots': r'vitesse|rapide|trop vite|ralentir', 'urgence': 3, 'facilite': 2},
    'Violence routière': {'mots': r'violence|insulte|intimidation|agression', 'urgence': 3, 'facilite': 2},
    'Entretien pistes': {'mots': r'entretien|entretenir|nid de poule|trou|verre', 'urgence': 2, 'facilite': 1},
    'Stationnement gênant': {'mots': r'stationnement|garé|voiture|parking|double file', 'urgence': 2, 'facilite': 2},
    'Stationnement vélo': {'mots': r'stationnement vélo|parking vélo|garage vélo', 'urgence': 2, 'facilite': 1},
    'Limitation trafic': {'mots': r'limiter trafic|réduire trafic|moins de voiture', 'urgence': 2, 'facilite': 3},
    'Signalisation': {'mots': r'panneau|feu|signalisation|marquage', 'urgence': 1, 'facilite': 1},
    'Conflit piétons': {'mots': r'piéton|trottoir|partagé|quai', 'urgence': 1, 'facilite': 2},
}

def get_label_facilite(score):
    if score <= 1.5: return "🟢 Facile"
    if score <= 2.5: return "🟡 Moyen"
    return "🔴 Difficile"

# --- 2. FONCTIONS D'ANALYSE ---
def detecter_themes(texte):
    if pd.isna(texte) or texte == "": return []
    return [nom for nom, cfg in CONFIG_PROBLEMES.items() if re.search(cfg['mots'], str(texte).lower())]

# --- 3. CHARGEMENT ET TRAITEMENT ---
print(" Chargement des données...")
df = pd.read_csv('reponses_questionnaire_lyon.csv', low_memory=False)

# Fusion des textes et détection
df['verbatim'] = df[['q34_texte', 'q35', 'q38_texte']].fillna('').agg(' '.join, axis=1)
df['problemes_list'] = df['verbatim'].apply(detecter_themes)

# --- 4. ANALYSE PAR TYPE DE PROBLÈME (FACILITÉ GÉNÉRALE) ---
print("\n ANALYSE DE LA FACILITÉ PAR TYPE DE PROBLÈME")
all_p = [p for sub in df['problemes_list'] for p in sub]
counts_global = Counter(all_p)

analyse_p = []
for nom, cfg in CONFIG_PROBLEMES.items():
    analyse_p.append({
        'Problème': nom,
        'Citations': counts_global[nom],
        'Urgence': cfg['urgence'],
        'Facilité': cfg['facilite'],
        'Type': get_label_facilite(cfg['facilite'])
    })

df_facilite_theme = pd.DataFrame(analyse_p).sort_values(by='Facilité')
print(df_facilite_theme.to_string(index=False))

# --- 5. ANALYSE PAR COMMUNE (TOP 10 + FACILITÉ MOYENNE) ---
print("\n ANALYSE PAR COMMUNE (TOP 10 ET FAISABILITÉ)")

resultats_communes = []
for insee, group in df.groupby('insee'):
    # Extraction de tous les problèmes de la commune
    commune_probs = [p for sub in group['problemes_list'] for p in sub]
    top_10_list = Counter(commune_probs).most_common(10)
    
    if top_10_list:
        # Calcul de la facilité moyenne des problèmes cités dans cette commune
        moy_facilite = np.mean([CONFIG_PROBLEMES[p]['facilite'] for p, _ in top_10_list])
        top_10_str = " | ".join([f"{p} ({c})" for p, c in top_10_list])
        
        resultats_communes.append({
            'INSEE': insee,
            'Réponses': len(group),
            'Sécurité_Moy': round(group['q14'].mean(), 2),
            'Facilité_Action': get_label_facilite(moy_facilite),
            'Top_10_Problemes': top_10_str
        })

df_communes = pd.DataFrame(resultats_communes).sort_values(by='Réponses', ascending=False)

# --- 6. AFFICHAGE FINAL ---
pd.set_option('display.max_colwidth', 100)
print("\n" + "="*120)
print(df_communes.head(15).to_string(index=False))
print("="*120)

 Chargement des données...

 ANALYSE DE LA FACILITÉ PAR TYPE DE PROBLÈME
                Problème  Citations  Urgence  Facilité        Type
      Stationnement vélo         32        2         1    🟢 Facile
        Entretien pistes       5754        2         1    🟢 Facile
           Signalisation        465        1         1    🟢 Facile
       Violence routière       7171        3         2     🟡 Moyen
    Stationnement gênant       5199        2         2     🟡 Moyen
       Vitesse excessive       9720        3         2     🟡 Moyen
         Conflit piétons       6363        1         2     🟡 Moyen
    Carrefours dangereux        348        3         3 🔴 Difficile
       Limitation trafic         11        2         3 🔴 Difficile
Infrastructure manquante      10169        3         3 🔴 Difficile

 ANALYSE PAR COMMUNE (TOP 10 ET FAISABILITÉ)

 INSEE  Réponses  Sécurité_Moy Facilité_Action                                                                                                 

In [195]:
!pip install geopandas
!pip install osmnx geopandas
!pip install scipy scikit-learn
!pip install pyspark
!pip install pyarrow	

     ---------------------------------------- 0.0/455.3 MB ? eta -:--:--
     --------------------------------------- 2.9/455.3 MB 27.9 MB/s eta 0:00:17
      -------------------------------------- 6.3/455.3 MB 21.5 MB/s eta 0:00:21
      ------------------------------------- 10.0/455.3 MB 20.7 MB/s eta 0:00:22
     - ------------------------------------ 13.1/455.3 MB 18.7 MB/s eta 0:00:24
     - ------------------------------------ 23.1/455.3 MB 25.6 MB/s eta 0:00:17
     -- ----------------------------------- 30.9/455.3 MB 28.0 MB/s eta 0:00:16
     --- ---------------------------------- 39.3/455.3 MB 30.1 MB/s eta 0:00:14
     --- ---------------------------------- 45.1/455.3 MB 29.6 MB/s eta 0:00:14
     ---- --------------------------------- 50.6/455.3 MB 29.3 MB/s eta 0:00:14
     ---- --------------------------------- 55.6/455.3 MB 28.8 MB/s eta 0:00:14
     ---- --------------------------------- 59.8/455.3 MB 28.0 MB/s eta 0:00:15
     ----- -------------------------------- 64.

In [196]:
import geopandas as gpd
from shapely.geometry import Point

# 1. Lecture du fichier CSV avec Pandas
file_path = "points_rouges_lyon_complet.csv"
df = pd.read_csv(file_path)

# 2. Conversion du DataFrame en GeoDataFrame
# On crée la colonne 'geometry' à partir des colonnes longitude et latitude
gdf = gpd.GeoDataFrame(
    df, 
    geometry=gpd.points_from_xy(df.longitude, df.latitude),
    crs="EPSG:4326"  # Système de coordonnées standard (WGS84)
)

# Affichage des premières lignes
print(f"Nombre de lignes : {len(gdf)}")
print(f"Type de géométrie : {gdf.geometry.type.unique()}")
display(gdf.head())

Nombre de lignes : 24256
Type de géométrie : ['Point']


,longitude,latitude,commune_insee,description,geometry
0,4.836445,45.758438,69123,NaN,POINT (4.83645 45.75844)
1,4.859543,45.731105,69123,NaN,POINT (4.85954 45.7311)
2,4.833717,45.731325,69123,NaN,POINT (4.83372 45.73133)
3,4.867707,45.729358,69123,NaN,POINT (4.86771 45.72936)
4,4.797329,45.803997,69194,Éboulement,POINT (4.79733 45.804)


In [197]:
import osmnx as ox
import geopandas as gpd
import pandas as pd

# 1. Charger le réseau routier de Lyon
# "drive" récupère toutes les rues carrossables
G = ox.graph_from_place("Lyon, France", network_type="drive")

# 2. Projeter le graphe en mètres (UTM) pour un calcul de distance précis
G_proj = ox.project_graph(G)

# 3. Préparer vos données (gdf) 
# On s'assure qu'elles sont dans le même système de coordonnées que le graphe projeté
gdf_proj = gdf.to_crs(G_proj.graph['crs'])

# 4. Trouver l'arête (rue) la plus proche pour chaque point
# ox.nearest_edges renvoie l'ID de la rue (u, v, key)
nearest_edges = ox.nearest_edges(G_proj, X=gdf_proj.geometry.x, Y=gdf_proj.geometry.y)

# 5. Calculer le point exact sur la rue (projection orthogonale)
# On utilise la fonction interpolate sur la géométrie de la rue
edges = ox.graph_to_gdfs(G_proj, nodes=False)
projected_points = []

for i, (point, edge_id) in enumerate(zip(gdf_proj.geometry, nearest_edges)):
    # Récupérer la ligne (geometry) de la rue correspondante
    street_line = edges.loc[edge_id, 'geometry']
    # Trouver le point sur cette ligne le plus proche de notre point d'origine
    snap_point = street_line.interpolate(street_line.project(point))
    projected_points.append(snap_point)

# 6. Créer un GeoDataFrame temporaire pour convertir les points projetés en Lat/Lon
snapped_gdf = gpd.GeoDataFrame(geometry=projected_points, crs=G_proj.graph['crs'])
snapped_gdf = snapped_gdf.to_crs(epsg=4326) # Retour en format GPS standard

# 7. Ajouter les colonnes au DataFrame d'origine
gdf['longitude_modified'] = snapped_gdf.geometry.x
gdf['latitude_modified'] = snapped_gdf.geometry.y

# Affichage des résultats
display(gdf[['longitude_modified', 'latitude_modified']].head())

,longitude_modified,latitude_modified
0,4.837045,45.758318
1,4.859541,45.731103
2,4.833732,45.731342
3,4.867713,45.729350
4,4.796513,45.791578
